# Population in districts

In [ ]:
import geopandas as gpd
import pandas as pd

## Load data from file

In [ ]:
warsaw_population = pd.read_json('data/warsaw_population.json')
warsaw_population

In [ ]:
warsaw_districts = gpd.read_file('data/warsaw_districts.geojson')
warsaw_districts

## Join data

In [ ]:
warsaw_districts_with_population = warsaw_districts.merge(warsaw_population, on='district', how='inner')
warsaw_districts_with_population

## Show data on the map

In [ ]:
warsaw_districts_with_population.explore('population')

# Population in buildings

## Load data from OpenStreetMap

In [ ]:
from srai.loaders.osm_loaders import OSMOnlineLoader

In [ ]:
warsaw_region = warsaw_districts_with_population.unary_union

In [ ]:
loader = OSMOnlineLoader()

osm_building_types = [
    "residential",
    "apartments",
    "house",
    "semidetached_house",
    "detached",
]

buildings = loader.load(
    gpd.GeoDataFrame(geometry=[warsaw_region], crs="EPSG:4326"),
    # warsaw_region,
    {"building": osm_building_types, "building:levels": True},
)
buildings

In [ ]:
buildings = buildings[buildings["building"].isin(osm_building_types)]
buildings = buildings.fillna(1)
buildings

## Clear data

In [ ]:
def map_floors(floors_value: str) -> int:
    try:
        floors = int(floors_value)
    except Exception:
        floors = 1

    return floors

In [ ]:
buildings["building:levels"] = buildings["building:levels"].map(map_floors)
buildings

## Interpolate population over buildings
Use number of floors and area of a building as a weight - bigger and higher building usually have more inhabitants

In [ ]:
buildings['weight'] = buildings.area * buildings['building:levels']
buildings

In [ ]:
xmin = 21.042753111534097
xmax = 21.069257679735955
ymin = 52.24187245384607
ymax = 52.22533280016626
buildings.cx[xmin:xmax, ymin:ymax].explore()

In [ ]:
buildings["geometry"] = buildings.centroid
buildings.cx[xmin:xmax, ymin:ymax].explore()

In [ ]:
buildings_with_districts = buildings.sjoin(warsaw_districts_with_population, predicate='within')
buildings_with_districts

In [ ]:
total_weight_in_districts = buildings_with_districts.groupby('district')['weight'].sum().rename("total_weight")
total_weight_in_districts

In [ ]:
buildings_with_population = buildings_with_districts.merge(total_weight_in_districts, on="district", how="inner")
buildings_with_population["population_in_building"] = (
    buildings_with_population["population"] * buildings_with_population["weight"] / buildings_with_population["total_weight"]
).round()
buildings_with_population

## Aggregate data into H3 index for easier representation

In [ ]:
import h3

buildings_with_population["h3"] = buildings_with_population.apply(
    lambda row: h3.latlng_to_cell(row.geometry.y, row.geometry.x, 8), axis=1
)
buildings_with_population

In [ ]:
population_aggregated = (
    buildings_with_population.groupby("h3")["population_in_building"]
    .sum()
    .rename("population_in_h3")
    .reset_index()
)
population_aggregated

In [ ]:
population_aggregated.population_in_h3.sum()

In [ ]:
from srai.h3 import h3_to_geoseries
population_aggregated = gpd.GeoDataFrame(population_aggregated, geometry=h3_to_geoseries(population_aggregated.h3))
population_aggregated

## Show data on the map
3 proposed solutions:
- simple GeoPandas `.explore()`
- little bit prettier `srai.plotting` solution
- 3D Deck.gl map

In [ ]:
# base folium - explore
population_aggregated.explore('population_in_h3')

In [ ]:
# srai - plot numeric
from srai.plotting import plot_numeric_data

plot_numeric_data(
    population_aggregated.rename(columns={"h3": "region_id"}).set_index("region_id"),
    "population_in_h3",
)

In [ ]:
# pydeck 3d
from srai.plotting.folium_wrapper import _generate_linear_colormap
import plotly.express as px
import pydeck as pdk

colormap = _generate_linear_colormap(
    # https://plotly.com/python/builtin-colorscales/
    px.colors.sequential.Aggrnyl_r,
    min_value=population_aggregated["population_in_h3"].min(),
    max_value=population_aggregated["population_in_h3"].max(),
)

population_aggregated["color"] = population_aggregated["population_in_h3"].map(
    colormap.rgb_bytes_tuple
)

# Define a layer to display on a map
layer = pdk.Layer(
    "H3HexagonLayer",
    population_aggregated,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=True,
    get_hexagon="h3",
    get_fill_color="[color[0], color[1], color[2], 204]",
    elevation_scale=0.5,
    get_elevation="population_in_h3",
    coverage=0.8,
)

# Set the viewport location
view_state = pdk.ViewState(
    latitude=52.2317, longitude=21.0062, zoom=9.5, bearing=0, pitch=30
)


# Render
pdk.Deck(
    layers=[layer],
    map_style="light",
    initial_view_state=view_state,
    tooltip={"text": "Population: {population_in_h3}"},
)

# OneBoxes by Allegro
Find out which one is the greenest

In [ ]:
one_boxes_raw = pd.read_json('data/oneboxes.json')
one_boxes_raw

In [ ]:
one_boxes_gdf = gpd.GeoDataFrame(
    one_boxes_raw,
    geometry=gpd.GeoSeries.from_xy(one_boxes_raw["lon"], one_boxes_raw["lat"]),
    crs="EPSG:4326",
)
one_boxes_gdf.explore()

In [ ]:
warsaw_one_boxes = one_boxes_gdf.clip(warsaw_region)
warsaw_one_boxes.explore()

In [ ]:
from srai.geometry import buffer_geometry

In [ ]:
warsaw_one_boxes = warsaw_one_boxes.rename(columns={"geometry": "point"})
warsaw_one_boxes["buffer_1000m"] = warsaw_one_boxes["point"].apply(
    lambda geometry: buffer_geometry(geometry, 1000)
)
warsaw_one_boxes = warsaw_one_boxes.set_geometry("buffer_1000m")
warsaw_one_boxes

In [ ]:
warsaw_one_boxes.explore(style_kwds=dict(opacity=0.5, fillOpacity=0.1))

In [ ]:
from srai.loaders.osm_loaders import OSMPbfLoader

In [ ]:
loader = OSMPbfLoader()

In [ ]:
greenery = loader.load(
    gpd.GeoDataFrame(geometry=[warsaw_region], crs="EPSG:4326"),
    # warsaw_region,
    {
        "leisure": ["garden", "park"],
        "natural": ["wood", "scrub", "heath", "grassland"],
        "landuse": ["grass", "orchard", "flowerbed", "forest", "greenfield", "meadow"],
    },
)
greenery

In [ ]:
greenery = greenery[greenery.geom_type != 'Point']
greenery

In [ ]:
greenery.plot(color="tab:green")

In [ ]:
warsaw_one_boxes_with_greenery = warsaw_one_boxes.sjoin(greenery)
warsaw_one_boxes_with_greenery

In [ ]:
warsaw_one_boxes_with_greenery = gpd.overlay(
    warsaw_one_boxes,
    greenery,
    how="intersection",
)

warsaw_one_boxes_with_greenery

In [ ]:
# Dissolve replaces groupby in spatial operations
# https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html
warsaw_one_boxes_with_greenery = warsaw_one_boxes_with_greenery.dissolve(by='id')
warsaw_one_boxes_with_greenery

In [ ]:
warsaw_one_boxes_with_greenery.plot(color='tab:green')

In [ ]:
warsaw_one_boxes_with_greenery['greenery_area'] = warsaw_one_boxes_with_greenery['geometry'].area
warsaw_one_boxes_with_greenery = warsaw_one_boxes_with_greenery.sort_values(by='greenery_area', ascending=False)
warsaw_one_boxes_with_greenery.head()

In [ ]:
m = warsaw_one_boxes.merge(
    warsaw_one_boxes_with_greenery["greenery_area"].reset_index(), on="id"
).explore(
    "greenery_area",
    cmap="BuGn",
    tiles="CartoDB positron",
    style_kwds=dict(opacity=0.5, fillOpacity=0.1),
)

warsaw_one_boxes['point'].explore(m=m, color='green')

In [ ]:
warsaw_one_boxes_with_greenery.loc[["AL014WXY"]].explore(color="green")

# Venturilo - Contextual count embedder 

In [ ]:
stations_raw = pd.read_json('data/venturilo_stations.json')
stations_raw

In [ ]:
stations_gdf = gpd.GeoDataFrame(
    stations_raw,
    geometry=gpd.GeoSeries.from_xy(stations_raw["lon"], stations_raw["lat"]),
    crs="EPSG:4326",
)
stations_gdf.explore()

In [ ]:
from srai.regionalizers import H3Regionalizer
from srai.plotting import plot_regions

In [ ]:
h3_regions = H3Regionalizer(resolution=9).transform(
    gpd.GeoDataFrame(geometry=[warsaw_region], crs="EPSG:4326")
    # warsaw_region
)
plot_regions(h3_regions)

In [ ]:
from srai.loaders.osm_loaders.filters import GEOFABRIK_LAYERS

In [ ]:
geofabrik_features = OSMPbfLoader().load(
    gpd.GeoDataFrame(geometry=[warsaw_region], crs="EPSG:4326"),
    # warsaw_region,
    GEOFABRIK_LAYERS
)
geofabrik_features

In [ ]:
geofabrik_features_without_stations = geofabrik_features[
    geofabrik_features["shopping"] != "amenity=bicycle_rental"
]
geofabrik_features_without_stations

In [ ]:
from srai.embedders import ContextualCountEmbedder
from srai.neighbourhoods import H3Neighbourhood
from srai.joiners import IntersectionJoiner

In [ ]:
joined_features = IntersectionJoiner().transform(h3_regions, geofabrik_features_without_stations)
joined_features

In [ ]:
embedder = ContextualCountEmbedder(neighbourhood=H3Neighbourhood(), neighbourhood_distance=5, concatenate_vectors=True)
embeddings = embedder.transform(regions_gdf=h3_regions, features_gdf=geofabrik_features_without_stations, joint_gdf=joined_features)
embeddings

In [ ]:
bikes_joint = IntersectionJoiner().transform(h3_regions, stations_gdf)
bikes_joint

In [ ]:
positive_samples = h3_regions.join(bikes_joint, how="inner")
positive_samples = positive_samples.reset_index().drop(columns=["feature_id"]).set_index("region_id")
positive_samples["is_positive"] = True
len(positive_samples)

In [ ]:
negative_samples = h3_regions.copy()
negative_samples["is_positive"] = False
negative_samples.loc[positive_samples.index, "is_positive"] = True
negative_samples = negative_samples[~negative_samples["is_positive"]]
len(negative_samples)

In [ ]:
negative_undersampled = negative_samples.sample(n=3 * len(positive_samples), random_state=42)
negative_undersampled

In [ ]:
CB_SAFE_PALLETE = [
    "#377eb8",
    "#ff7f00",
    "#4daf4a",
    "#f781bf",
    "#a65628",
    "#984ea3",
    "#999999",
    "#e41a1c",
    "#dede00",
]

In [ ]:
train_data = pd.concat([positive_samples, negative_undersampled])
train_data.explore("is_positive", cmap=CB_SAFE_PALLETE, zoom_start=14, height=600, tiles="CartoDB positron")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [ ]:
X = embeddings.loc[train_data.index].to_numpy()
y = train_data["is_positive"].astype(int).to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
classifier = SVC(probability=True)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
y_pred_proba = classifier.predict_proba(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
station_probas = classifier.predict_proba(embeddings.to_numpy())

h3_regions["station_proba"] = station_probas[:, 1]
m = plot_numeric_data(h3_regions, "station_proba", colormap="Spectral_r", opacity=0.5)

stations_gdf.explore(m=m, color='black')

## Transfer learning

In [ ]:
from srai.regionalizers import geocode_to_region_gdf

wroclaw_region = geocode_to_region_gdf('Wrocław, PL')
wroclaw_region

In [ ]:
wroclaw_h3_regions = H3Regionalizer(resolution=9).transform(wroclaw_region)
wroclaw_h3_regions

In [ ]:
wroclaw_geofabrik_features = OSMPbfLoader().load(
    wroclaw_region,
    GEOFABRIK_LAYERS
)
wroclaw_geofabrik_features_without_stations = wroclaw_geofabrik_features[
    wroclaw_geofabrik_features["shopping"] != "amenity=bicycle_rental"
]

In [ ]:
wroclaw_joined_features = IntersectionJoiner().transform(wroclaw_h3_regions, wroclaw_geofabrik_features_without_stations)

In [ ]:
wroclaw_embeddings = embedder.transform(
    regions_gdf=wroclaw_h3_regions,
    features_gdf=wroclaw_geofabrik_features_without_stations,
    joint_gdf=wroclaw_joined_features,
)
wroclaw_embeddings

In [ ]:
station_probas = classifier.predict_proba(wroclaw_embeddings.to_numpy())

wroclaw_h3_regions["station_proba"] = station_probas[:, 1]
plot_numeric_data(wroclaw_h3_regions, "station_proba", colormap="Spectral_r", opacity=0.5)

# Taxis and flow prediction

In [ ]:
# TODO